# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

In [3]:
MODEL = "llama3.2"

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
def get_links_with_ollama(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ]
    )
    
    result = response['message']['content']
    print(type(result))
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

edwarddonner = Website("https://edwarddonner.com")
edwarddonner.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [13]:
get_links_with_ollama("https://edwarddonner.com")

<class 'str'>


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'outsmart link', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'about me and about nebula link',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'posts link', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'careers/jobs page',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter link', 'url': 'https://twitter.com/edwarddonner'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_with_ollama(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://edwarddonner.com"))

<class 'str'>
Found links: {'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'company page', 'url': 'https://edwarddonner.com/'}, {'type': 'careers page', 'url': 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/'}]}
Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'company/career pages (repeated, assume unique link)', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


'You are looking at a company called: EdwarDdonner\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m pr

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
def create_brochure_with_ollama(company_name, url):
    response = ollama.chat(
        model=MODEL,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ],
    )
    
    result = response['message']['content']
    return result
    # display(Markdown(result))

In [21]:
result = create_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")
display(Markdown(result))

<class 'str'>
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'home page', 'url': 'https://edwarddonner.com/'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'}, {'type': 'Facebook profile', 'url': 'https://www.facebook.com/edward.donner.52'}]}


**Edward Donner Brochure**
==========================

**About Us**
-------------

At Edward Donner, we're passionate about harnessing the power of Artificial Intelligence (AI) to help people discover their potential and pursue their reason for being. Our mission is to make a positive impact on human prosperity by providing innovative solutions that connect talent with fulfilling roles.

**Our Story**
--------------

Edward Donner was founded by Ed, a co-founder and CTO of Nebula.io, an AI-powered recruitment platform. With over 20 years of experience in the financial services industry, Ed has seen firsthand the challenges of finding meaningful work for individuals. He's dedicated to applying AI to real-world problems, particularly in the field of hiring.

**Our Solution**
----------------

Edward Donner offers a patented matching model that connects people with roles that align with their strengths and interests. Our award-winning platform uses Generative AI and machine learning to provide accurate and efficient talent sourcing solutions for recruiters.

**Key Features**

*   **Patented Matching Model**: Connects people with roles without relying on keywords
*   **Generative AI**: Uses machine learning to improve matching accuracy and speed
*   **Ikigai Concept**: Helps individuals find roles that align with their passions and values

**Impact**
----------

At Edward Donner, we believe that everyone deserves to find a fulfilling career. With 77% of people feeling uninspired or disconnected at work, our mission is to raise the level of human prosperity by providing innovative solutions.

**Join Our Team**
-----------------

If you're passionate about AI and making a positive impact on society, join our team! We're always looking for talented individuals who share our vision. Check out our career page for current openings.

**Get in Touch**
---------------

Want to connect with Ed or learn more about Edward Donner? Follow us on LinkedIn, Twitter, Facebook, or subscribe to our newsletter.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
def stream_brochure_with_ollama(company_name, url):
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.get("message", {}).get("content", "")
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Company page', 'url': 'https://edwarddonner.com/'}]}


**Edward Donner Brochure**
==========================

**Welcome to Edward Donner**

We are a pioneering company in the field of Artificial Intelligence (AI), dedicated to helping people discover their potential and pursue their reason for being. Our innovative solutions use Generative AI and machine learning to match people with roles that bring fulfillment and success.

**Our Story**

Founded by Ed, our co-founder and CTO, Edward Donner has its roots in an AI startup called untapt, which he founded in 2013. After a 20-year career in Financial Services, Ed saw the potential of AI in real-world problems faced by everyone. He led untapt to significant success, including being voted the "startup most likely to grow exponentially" at an Amazon pitch event.

In 2021, untapt was acquired by GQR's parent company, and shortly afterwards, Nebula.io was born. Ed continued to drive innovation in AI-powered talent solutions, with a focus on creating platforms that make a positive impact on people's lives.

**Our Mission**

Our long-term goal is to help people find roles where they will be most fulfilled and successful, motivated by the concept of Ikigai (reason for being). We believe that by applying AI to real-world problems, we can raise the level of human prosperity.

**What We Do**

* Provide innovative talent solutions using Generative AI and machine learning
* Match people with roles that bring fulfillment and success
* Help people discover their potential and pursue their reason for being

**Our Values**

* **Innovation**: We are committed to staying at the forefront of AI innovation and solving real-world problems.
* **Empowerment**: We believe in empowering people to find their purpose and achieve their goals.
* **Collaboration**: We work closely with our customers, partners, and team members to drive success.

**Join Our Journey**

If you're passionate about AI, innovation, and making a positive impact on people's lives, we want to connect with you. Join us for a virtual coffee or real coffee (if you're in NYC!) to discuss how you can contribute to our mission.

**Get in Touch**

Email: [ed at] edwarddonner [dot] com
Website: www.edwarddonner.com
LinkedIn | Twitter | Facebook

Subscribe to our newsletter to stay up-to-date on the latest news and updates from Edward Donner.

In [25]:
translator_system_prompt = "You are a professional translator. Your task is to translate the provided company brochure from English to Vietnamese. \
Preserve the original structure and formatting, including any markdown sections, headings, or bullet points. \
Use natural, fluent, and professional Vietnamese suitable for prospective customers, investors, and job seekers in Vietnam. \
Avoid literal word-by-word translation — prioritize clarity, accuracy, and natural flow in Vietnamese."

In [26]:
def get_translation_user_prompt(brochure_content):
    user_prompt = "Please translate the following company brochure into Vietnamese.\n"
    user_prompt += "Preserve all formatting and structure (e.g., markdown headings, lists, etc).\n"
    user_prompt += "Use natural, fluent, and professional Vietnamese suitable for prospective customers, investors, and job seekers.\n"
    user_prompt += "\nBrochure content to translate:\n"
    user_prompt += brochure_content
    user_prompt = user_prompt[:5_000]  # Truncate if needed
    return user_prompt

In [27]:
def stream_translated_brochure_with_ollama(company_name, url):
    brochure_content = create_brochure_with_ollama(company_name, url)
    stream = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": translator_system_prompt},
            {"role": "user", "content": get_translation_user_prompt(brochure_content)},
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.get("message", {}).get("content", "")
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

In [28]:
stream_translated_brochure_with_ollama("EdwarDdonner", "https://edwarddonner.com")

<class 'str'>
Found links: {'links': [{'type': 'About page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'Company homepage', 'url': 'https://edwarddonner.com/'}, {'type': 'Careers/Jobs page', 'url': 'https://www.linkedin.com/in/eddonner/'}]}


# Brochure Edward Donner
==========================

**Mục Đích**
-------------

Tại Edward Donner, chúng tôi đang trên hành trình giúp mọi người khám phá tiềm năng của mình và theo đuổi lý do tồn tại của họ. Chúng tôi tin rằng trí tuệ nhân tạo có thể tạo ra một tác động tích cực to lớn khi được ứng dụng vào lĩnh vực khám phá tài năng.

**Về Chúng Tôi**
----------------

Chúng tôi là một nhóm người đam mê chia sẻ một mục tiêu chung: đổi mới cách các nhà tuyển dụng tìm kiếm, hiểu biết, tương tác và quản lý tài năng. Mô hình khớp kết hợp trí tuệ nhân tạo và các hình thức học máy khác sử dụng bằng Generative AI để khớp người với vai trò với độ chính xác và tốc độ cao hơn bất kỳ cái nào được tưởng tượng trước đây.

**Câu Chuyện Của Chúng Tôi**
---------------------------

Kinh nghiệm của Edward Donner bắt nguồn từ năm 2013 khi người sáng lập, Ed, thành lập một startup trí tuệ nhân tạo có tên là untapt. Công ty xây dựng các thị trường tài năng và phần mềm khoa học dữ liệu cho các công ty tuyển dụng, chuyên về các vai trò công nghệ trong lĩnh vực tài chính. Sau khi được chọn làm một chương trình khởi nghiệp uy tín và nhận được sự công nhận từ các tờ báo hàng đầu, untapt đã được mua lại bởi công ty mẹ của GQR năm 2021.

Từ thành công này, Nebula.io đã sinh ra, thực hiện việc ứng dụng trí tuệ nhân tạo vào các vấn đề thực tế, đặc biệt là trong lĩnh vực tuyển dụng. Mục tiêu dài hạn của chúng tôi là giúp mọi người tìm được những vai trò sẽ làm họ hạnh phúc và thành công nhất, theo ý thức về Ikigai.

**Sự Phổ Biến Của Chúng Tôi**
------------------------------

* **Hợp tác**: Chúng tôi tin rằng sự đa dạng các quan điểm và chuyên môn là cần thiết để thúc đẩy đổi mới.
* **Đổi Mới**: Chúng tôi luôn tìm kiếm những cách cải thiện sản phẩm và dịch vụ của mình bằng các công nghệ tiên tiến nhất như trí tuệ nhân tạo Generative AI.
* **Sáng Tạo**: Đội ngũ của chúng tôi được động viên bởi một cảm hứng chung về mục đích và mong muốn làm ra tác động tích cực.

**Khung Thể: Outsmart**
----------------------

Trông xem cách LLMs tham gia vào một trận chiến ngoại hình và tinh thần. Khung thể Outsmart thể hiện khả năng khớp kết hợp của mô hình trí tuệ nhân tạo độc quyền của chúng tôi, đặt các LLM chống lại nhau trong một thử thách về trí thông minh.

**Đính Hợp**
--------------

Nếu bạn quan tâm đến sứ mệnh và giá trị của Edward Donner, hoặc nếu bạn muốn kết nối với chúng tôi để uống cà phê trực tuyến, xin đừng ngần ngại liên hệ. Hãy cùng nhau đổi mới thế giới của khám phá tài năng!

### Liên Hệ

Email: <ed [at] edwarddonner [dot] com>
Trang web: www.edwarddonner.com
LinkedIn: LinkedIn
Twitter: Twitter
Facebook: Facebook

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>